<a href="https://colab.research.google.com/github/Xanthor-Aditya/Pytorch-Practice_Codes/blob/master/Cifar10-Conv2d-pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
use_cuda = True

In [0]:
import torch

In [0]:
import torch.nn as nn

In [0]:
import torch.nn.functional as F

In [0]:
import torchvision.datasets as dsets

In [0]:
import torchvision.transforms as transforms

In [0]:
from torch.autograd import Variable

In [0]:
input_size = (3,32,32)


In [0]:
number_of_filters1 = 16
filter_size1 = 3
number_of_filters2 = 32
filter_size2 = 5
padding = 1
stride = 1
hidden_size = 128
num_classes = 10
num_epochs = 20
batch_size = 32
learning_rate = 0.0001
nn_input= 900

In [9]:
train_dataset = dsets.CIFAR10(root='./data',
                           train=True,
                           transform= transforms.ToTensor(),
                           download=True)

test_dataset = dsets.CIFAR10(root='./data',
                           train=False,
                           transform= transforms.ToTensor())

0it [00:00, ?it/s]

170500096it [00:07, 23120539.88it/s]                               


In [0]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [0]:
class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, number_of_filters1, filter_size1,stride, padding)
    self.conv2 = nn.Conv2d(number_of_filters1, number_of_filters2, filter_size2, stride, padding)
    self.bn1 = nn.BatchNorm2d(number_of_filters2)
    #self.global_mxpool = nn.AvgPool2d(2,2)
    self.fc1 = nn.Linear(nn_input*number_of_filters2, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)
  def forward(self,x):
    out = F.relu(self.conv1(x))
    out = self.conv2(out)
    out = F.relu(self.bn1(out))
    out = out.view(-1, nn_input*number_of_filters2)
    #out = self.global_mxpool(out)
    out = self.fc1(out)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [0]:
net = Net(input_size, hidden_size, num_classes)
criterian = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate)

In [0]:
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):    
    images = Variable(images)
    labels = Variable(labels)
    optimizer.zero_grad()
    outputs = net(images)
    loss = criterian(outputs, labels)
    loss.backward()
    optimizer.step()
    if (i+1)%100 == 0:
          print('Epoch [%d/%d], Step [%d/%d],Loss: %.4f'
                %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size,loss.data))
      
    

In [14]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images)    
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1) 
    total += labels.size(0)               
    correct += (predicted == labels).sum()
    
print('Accuracy on 10k test images: %d %%' % (100 * correct / total))

Accuracy on 10k test images: 64 %
